# Helmet Mapping + Deepsort + Posit

In this notebook:
1. I introduce a package `helmet_assignment` which includes some helper code.
2. I show how `deepsort` can be applied to post process exsiting predictions.

Postprocessing is applied to the baseline predictions from [this amazing starter notebook](https://www.kaggle.com/its7171/nfl-baseline-simple-helmet-mapping).


# Helmet Mapping + Deepsort + Posit

In this notebook:
1. I introduce a package `helmet_assignment` which includes some helper code.
2. I show how `deepsort` can be applied to post process exsiting predictions.

Postprocessing is applied to the baseline predictions from [this amazing starter notebook](https://www.kaggle.com/its7171/nfl-baseline-simple-helmet-mapping).




# "helmet_assignment" package provides helper code.

This package includes helpful functions like`NFLAssignmentScorer`, `check_submission` and `add_track_features`.

- dataset: https://www.kaggle.com/robikscube/helmet-assignment-helpers
- github: https://github.com/RobMulla/helmet-assignment# "helmet_assignment" package provides helper code.

This package includes helpful functions like`NFLAssignmentScorer`, `check_submission` and `add_track_features`.

- dataset: https://www.kaggle.com/robikscube/helmet-assignment-helpers
- github: https://github.com/RobMulla/helmet-assignment

# Baseline helmet mapping
This section uses the simple helmet mapping approach from the awesome notebook:

https://www.kaggle.com/its7171/nfl-baseline-simple-helmet-mapping

In [3]:
# Install helmet-assignment helper code
# !pip install input/helmet-assignment-helpers/helmet-assignment-main/ > /dev/null 2>&1
from helmet_assignment.score import NFLAssignmentScorer, check_submission
from helmet_assignment.features import add_track_features

import numpy as np
import pandas as pd
import glob
import os
import cv2
import itertools
from sklearn.metrics import accuracy_score
from tqdm.auto import tqdm
from multiprocessing import Pool
from matplotlib import pyplot as plt
from sklearn.cluster import KMeans
import random
random.seed(42)
from matplotlib.pyplot import figure


## Settings and loading data
n_test_videos = len(os.listdir('input/nfl-health-and-safety-helmet-assignment/test/'))
# Run in debug mode unless during submission
if n_test_videos == 6:
    debug = True
else:
    debug = False



# Read in the data.

BASE_DIR = 'input/nfl-health-and-safety-helmet-assignment'

labels = pd.read_csv(f'{BASE_DIR}/train_labels.csv')
if debug:
    tracking = pd.read_csv(f'{BASE_DIR}/train_player_tracking.csv')
    helmets = pd.read_csv(f'{BASE_DIR}/train_baseline_helmets.csv')
else:
    tracking = pd.read_csv(f'{BASE_DIR}/test_player_tracking.csv')
    helmets = pd.read_csv(f'{BASE_DIR}/test_baseline_helmets.csv')
    
    
tracking = add_track_features(tracking)


def add_cols(df):
    df['game_play'] = df['video_frame'].str.split('_').str[:2].str.join('_')
    if 'video' not in df.columns:
        df['video'] = df['video_frame'].str.split('_').str[:3].str.join('_') + '.mp4'
    return df

helmets = add_cols(helmets)

In [4]:
#  Configurables
n_debug_samples = 2
random_state = 42
CONF_THRE = 0.3
max_iter = 1000
DIG_STEP = 3
DIG_MAX = DIG_STEP*10

if debug:
    helmets = add_cols(helmets)
    labels = add_cols(labels)
    # Select `n_debug_samples` worth of videos to debug with
#     sample_videos = labels['video'].drop_duplicates().sample(n_debug_samples, random_state=random_state).tolist()
    sample_videos = ['57783_003374_Endzone.mp4','57783_003374_Sideline.mp4']
    print('sample_videos',sample_videos)
    sample_gameplays = ['_'.join(x.split('_')[:2]) for x in sample_videos]
    print('sample_gameplays',sample_gameplays)
    tracking = tracking[tracking['game_play'].isin(sample_gameplays)]
    helmets = helmets[helmets['video'].isin(sample_videos)]
    labels = labels[labels['video'].isin(sample_videos)]
tracking.shape, helmets.shape, labels.shape

sample_videos ['57783_003374_Endzone.mp4', '57783_003374_Sideline.mp4']
sample_gameplays ['57783_003374', '57783_003374']


((6424, 18), (19412, 8), (16844, 15))

# Notebook #1

In [5]:
def find_nearest(array, value): # находит значение ближайшего est_frame-а к value (фрэйму)
    value = int(value)
    array = np.asarray(array).astype(int)
    idx = (np.abs(array - value)).argmin() # из номеров est_frame вычитаем frame, берем модуль. Находим индекс минимального элемента в массиве
    return array[idx] # значение ближайшего est_frame-а

def norm_arr(a): # Нормируем от 0 до 1 - вычитаем из каждого значения минимальное и делим каждое значение на максимальное (после вычета)
    a = a-a.min()
    a = a/a.max()
    return a
    
def dist(a1, a2): 
    '''берем разность между треком и шлемами и находим норму фробениуса 
    (квадратный корень сумм квадратов модулей элементов матрицы размера, 
    то есть в идеале, когда матрицы одинаковые, норма равна 0) '''
    return np.linalg.norm(a1-a2)

def dist_for_different_len(a1, a2): # Передаем координату X трекинга (rotated) и центры шлемов
    assert len(a1) >= len(a2), f'{len(a1)}, {len(a2)}' # проверем что треков больше или равно шлемов
    len_diff = len(a1) - len(a2) # количество 'лишних' треков
    a2 = norm_arr(a2) # func 2. Передаем центры шлемов. Нормируем от 0 до 1 - вычитаем из каждого значения минимальное и делим каждое значение на максимальное (после вычета)
    if len_diff == 0: # Если нет 'лишних' треков
        a1 = norm_arr(a1) # func 2. Передаем координату Х трека. Нормируем от 0 до 1
        return dist(a1,a2), () # func 3. Получаем разность между треком и шлемами и находим норму фробениуса (квадратный корень сумм квадратов модулей элементов матрицы размера. Равна 0, когда матрицы одинаковые)
        # возвращаем значение дистанции и пустоту (тут никакие треки не выкидываем)
        
    else: # если есть "лишние" треки без боксов
        min_dist = 10000
        min_detete_idx = None
        cnt = 0
        del_list = list(itertools.combinations(range(len(a1)),len_diff)) # combinations(range(4), 3) --> 012 013 023 123. Координаты X трека перебираем количеством лишних треков и делаем из всех вариантов список для перебора
        if len(del_list) > max_iter: # если длина этого списка для перебора больше макс количества итераций
            del_list = random.sample(del_list, max_iter) # то берем случайный семпл размера макс количества итераций
        for detete_idx in del_list: # перебираем список для перебора (айдишники на удаление)
            this_a1 = np.delete(a1, detete_idx) # убираем 'лишние' delete_idx из массива a1. Теперь количество треков уравнивается с количеством шлемов
            this_a1 = norm_arr(this_a1) # func 2. Передаем сокращенные координаты Х трека. Нормируем от 0 до 1
            this_dist = dist(this_a1, a2) # func 3. Получаем разность между треком и шлемами и находим норму фробениуса (квадратный корень сумм квадратов модулей элементов матрицы размера. Равна 0, когда матрицы одинаковые)
            #print(len(a1), len(a2), this_dist)
            if min_dist > this_dist: # оставляем вариант с минимальной дистанцией, запоминаем айдишники треков которые нужно будет выкинуть
                min_dist = this_dist
                min_detete_idx = detete_idx
                
        return min_dist, min_detete_idx # возвращаем значение минимальной дистанции и айдишники треков которые нужно будет выкинуть
        
def rotate_arr(u, t, deg=True): 
    '''ортогональное преобразование (всякое ортогональное преобразование является поворотом или поворотом с инверсией). 
    Ортогональные преобразования сохраняют длины векторов и углы между векторами'''
    if deg == True:
        t = np.deg2rad(t)
    # при нуле [1 0][0 1] при 90 градусах [0,-1][1, 0]
    R = np.array([[np.cos(t), -np.sin(t)],
                  [np.sin(t),  np.cos(t)]])
    return  np.dot(R, u) # shape (2, 2) (2, 22)


def dist_rot(tracking_df, a2): # Передаем трек и координаты упорядоченных горизонтальных центров шлемов
    tracking_df = tracking_df.sort_values('x')
    x = tracking_df['x']
    y = tracking_df['y']
    min_dist = 10000
    min_idx = None
    min_x = None
    dig_step = 3
    dig_max = dig_step*10
    for dig in range(-dig_max,dig_max+1,dig_step):
        arr = rotate_arr(np.array((x,y)), dig) # func 5. Берем координаты игроков (x,y) и поворачиваем их на угол из цикла. Shape (2,22) и (x,y) и arr
        this_dist, this_idx = dist_for_different_len(np.sort(arr[0]), a2) # func 4. Передаем координату X трекинга (rotated) и центры шлемов
        if min_dist > this_dist: # оставляем минимальные дистанцию и соответствующие ей айдишники на выброс
            min_dist = this_dist
            min_idx = this_idx
            min_x = arr[0] # оставляем только ось X
    tracking_df['x_rot'] = min_x # повернутое значение X
    player_arr = tracking_df.sort_values('x_rot')['player'].values # сортированное повернутое значение X
    players = np.delete(player_arr,min_idx) # удаляем айдишники на удаление и получаем список "обозначенных игроков"
    return min_dist, players # возвращаем значение нормы и "обозначенных игроков"



def mapping_df(args):
    video_frame, df = args # video_frame 57906_000718_Endzone_1, and dataframe helmets for this frame

    gameKey,playID,view,frame = video_frame.split('_') #     Разбиваем video_frame 57906_000718_Endzone_1 на четыре куска по "_"
    gameKey = int(gameKey)
    playID = int(playID)
    frame = int(frame)
    
    this_tracking = tracking[(tracking['gameKey']==gameKey) & (tracking['playID']==playID)] # отсекаем tracking для этой игры
    est_frame = find_nearest(this_tracking.est_frame.values, frame) # func 1 (номера 10032 кадров, из которых уникальных 456 и номер кадра). Находим ближайший кадр
    this_tracking = this_tracking[this_tracking['est_frame']==est_frame] # оставляем датасет ближайшего фрейма
    len_this_tracking = len(this_tracking) # его длина, по идее 22 (соответствует количеству игроков)
    
    # df-боксы этого кадра. Находим горизонтальные центры шлемов (положительный и отрицательный)
    df['center_h_p'] = (df['left']+df['width']/2).astype(int) 
    df['center_h_m'] = (df['left']+df['width']/2).astype(int)*-1
    df = df[df['conf']>CONF_THRE].copy() # оставляем боксы с conf больше threshold-а
    if len(df) > len_this_tracking: # если боксов больше чем треков, то отсекаем лучшие боксы (количество равно количеству треков)
        df = df.tail(len_this_tracking)
    df_p = df.sort_values('center_h_p').copy() # сортируем шлемы по горизонтальным центрам от меньшего значения к большему (видимо слева направо)
    df_m = df.sort_values('center_h_m').copy() # сортируем шлемы по горизонтальным центрам от меньшего значения к большему, но тут для отрицательных чисел (видимо справа налево)
    
    if view == 'Endzone': # если вид вдоль поля (по ТВ обычно показывают вид поперек поля, как в футболе). То меняем местами координаты x и y
        this_tracking['x'], this_tracking['y'] = this_tracking['y'].copy(), this_tracking['x'].copy()
    a2_p = df_p['center_h_p'].values # делаем pandas series из упорядоченных горизонтальных центров положительных
    a2_m = df_m['center_h_m'].values # делаем pandas series из упорядоченных горизонтальных центров отрицательных
    
    min_dist_p, min_detete_idx_p = dist_rot(this_tracking ,a2_p) # func 6. Передаем трек и координаты упорядоченных горизонтальных центров шлемов полож. Возвращаем значение нормы и "обозначенных игроков"
    min_dist_m, min_detete_idx_m = dist_rot(this_tracking ,a2_m) # func 6. Передаем трек и координаты упорядоченных горизонтальных центров шлемов отриц. Возвращаем значение нормы и "обозначенных игроков"
    
    # оставляем из положительного и отрицательного варианта наилучший
    if min_dist_p < min_dist_m:
        min_dist = min_dist_p
        min_detete_idx = min_detete_idx_p
        tgt_df = df_p
    else:
        min_dist = min_dist_m
        min_detete_idx = min_detete_idx_m
        tgt_df = df_m
        
    #print(video_frame, len(this_tracking), len(df), len(df[df['conf']>CONF_THRE]), this_tracking['x'].mean(), min_dist_p, min_dist_m, min_dist)
    tgt_df['label'] = min_detete_idx # "обозначенные игроки"
    return tgt_df[['video_frame','left','width','top','height','label']]

In [6]:
# p = Pool(processes=4)
# submission_df_list = []
# df_list = list(helmets.groupby('video_frame')) # итерируемся по кадрам 2664 штуки
# with tqdm(total=len(df_list)) as pbar:
#     for this_df in p.imap(mapping_df, df_list): # func 7
#         submission_df_list.append(this_df)
#         pbar.update(1)
# p.close()

# submission_df = pd.concat(submission_df_list)
# submission_df.to_csv('submission-baseline.csv', index=False)
# submission_df = pd.read_csv('submission-baseline.csv')

submission_df = pd.read_csv('input/baseline-seed-not-fixed/submission-baseline_57783_003374_E_and_S.csv')


## Score the predictions before applying deepsort postprocessing
# The scores are roughly ~0.3, which is similar to the public leaderboard.
if debug:
    scorer = NFLAssignmentScorer(labels)
    baseline_score = scorer.score(submission_df)
    print(f"validation score {baseline_score:0.4f}") # validation score 0.3121 # 0.2830 sample_videos ['57783_003374_Endzone.mp4', '57783_003374_Sideline.mp4']

validation score 0.2830


# Deepsort Postprocessing

Deepsort is a popular framework for object tracking within video. 
- [This blog post](https://nanonets.com/blog/object-tracking-deepsort/
) shows some examples of it being put to use.
- This notebook shows how to apply deepsort to this helmet dataset: https://www.kaggle.com/s903124/nfl-helmet-with-yolov5-deepsort-starter
- You can also read the paper for deepsort here: https://arxiv.org/pdf/1703.07402.pdf

The approach is fairly simple:
1. Step through each frame in a video and apply the deepsort algorithm. This clusters helmets across frames when it is the same player/helmet.
2. Group by each of these deepsort clusters - and pick the most common label for that cluster. Then override all of the predictions for that helmet to the same player.

## Importing Deepsort from dataset
Because your submission is not allowed to use internet access, you can reference the deepsort codebase from the attached dataset. Deepsort also has a dependency of `easydict` which I've also added as a dataset.

In [7]:
import sys
sys.path.append('input/easydict-master/easydict-master/')
# https://github.com/mikel-brostrom/Yolov5_DeepSort_Pytorch
sys.path.append('input/yolov5-deepsort-pytorch/Yolov5_DeepSort_Pytorch-master/Yolov5_DeepSort_Pytorch-master/deep_sort_pytorch/')
from deep_sort.deep_sort import DeepSort
from utils.parser import get_config


"""
Helper functions from yolov5 to plot deepsort labels.
"""

def compute_color_for_id(label):
    """
    Simple function that adds fixed color depending on the id
    """
    palette = (2 ** 11 - 1, 2 ** 15 - 1, 2 ** 20 - 1)

    color = [int((p * (label ** 2 - label + 1)) % 255) for p in palette]
    return tuple(color)

def plot_one_box(x, im, color=None, label=None, line_thickness=3):
    # Plots one bounding box on image 'im' using OpenCV
    assert im.data.contiguous, 'Image not contiguous. Apply np.ascontiguousarray(im) to plot_on_box() input image.'
    tl = line_thickness or round(0.002 * (im.shape[0] + im.shape[1]) / 2) + 1  # line/font thickness
    color = color or [random.randint(0, 255) for _ in range(3)]
    c1, c2 = (int(x[0]), int(x[1])), (int(x[2]), int(x[3]))
    cv2.rectangle(im, c1, c2, color, thickness=tl, lineType=cv2.LINE_AA)
    if label: 
        tf = max(tl - 1, 1)  # font thickness
        t_size = cv2.getTextSize(label, 0, fontScale=tl / 3, thickness=tf)[0]
        c2 = c1[0] + t_size[0], c1[1] - t_size[1] - 3
        cv2.rectangle(im, c1, c2, color, -1, cv2.LINE_AA)  # filled
        cv2.putText(im, label, (c1[0], c1[1] - 2), 0, tl / 3, [225, 255, 255], thickness=tf, lineType=cv2.LINE_AA)
    return im

In [8]:
%%writefile deepsort.yaml

DEEPSORT:
  REID_CKPT: "input/yolov5-deepsort-pytorch/ckpt.t7"
  MAX_DIST: 0.2
  MIN_CONFIDENCE: 0.3
  NMS_MAX_OVERLAP: 0.5
  MAX_IOU_DISTANCE: 0.7
  MAX_AGE: 70
  N_INIT: 3
  NN_BUDGET: 100

Writing deepsort.yaml


# Notebook #2 (Functions to apply deepsort to helmet boxes) & 3 (Alex)

### Base part

In [9]:
# Below are two functions `deepsort_helmets` which runs deepsort across a video. There is a lot of room for improving this function. 
# The merging of deepsort labels onto the original helmet boxes is currently done in a very crude manner.
# `add_deepsort_label_col` mapps the most common label to each deepsort cluster.


def add_deepsort_label_col(out): # 
    '''Даем кластеру лэйбл самого частого лэйбла. В результате добавляются label_deepsort (ИД игрока) и label_count_deepsort (число) #
    
    Input: датафрейм из deepsort_helmets - боксы, лэйблы и дипсорт кластеры
    Находим лэйбл, который в наибольшем количестве кадров присутствовал с определенным кластером. 
    
    3. Значение наибольшего количества записываем в label_count
    4. Названия кластеров из индекса переходят в столбец
    5-6. Группируем по ним и оставляем первый (самый частый)
    7-1. Получаем словарь соответствия кластера и самого частого лэйбла для этого кластера
    7-2. Получаем словарь соответствия кластера и значения этого самого частого лэйбла
    '''
    # Find the top occuring label for each deepsort_cluster 

    sortlabel_map = out.groupby('deepsort_cluster')['label'].value_counts() \
        .sort_values(ascending=False).to_frame() \
        .rename(columns={'label':'label_count'}) \
        .reset_index() \
        .groupby(['deepsort_cluster']) \
        .first()['label'].to_dict()
    # Find the # of times that label appears for the deepsort_cluster.
    sortlabelcount_map = out.groupby('deepsort_cluster')['label'].value_counts() \
        .sort_values(ascending=False).to_frame() \
        .rename(columns={'label':'label_count'}) \
        .reset_index() \
        .groupby(['deepsort_cluster']) \
        .first()['label_count'].to_dict()
    
    out['label_deepsort'] = out['deepsort_cluster'].map(sortlabel_map) # the top occuring label for each deepsort_cluster
    out['label_count_deepsort'] = out['deepsort_cluster'].map(sortlabelcount_map) # the # of times that label appears for the deepsort_cluster.

    return out

def score_vs_deepsort(myvideo, out, labels):
    # Score the base predictions compared to the deepsort postprocessed predictions.
    myvideo_mp4 = myvideo + '.mp4'
    labels_video = labels.query('video == @myvideo_mp4')
    scorer = NFLAssignmentScorer(labels_video)
    out_deduped = out.groupby(['video_frame','label']).first().reset_index()
    base_video_score = scorer.score(out_deduped)
    
    out_preds = out.drop('label', axis=1).rename(columns={'label_deepsort':'label'})
    print(out_preds.shape)
    out_preds = out_preds.groupby(['video_frame','label']).first().reset_index()
    print(out_preds.shape)
    deepsort_video_score = scorer.score(out_preds)
    print(f'{base_video_score:0.5f} before --> {deepsort_video_score:0.5f} deepsort')
    
    
def score_vs_posit(myvideo, out, labels):
    # Score the base predictions compared to the deepsort postprocessed predictions.
    myvideo_mp4 = myvideo + '.mp4'
    labels_video = labels.query('video == @myvideo_mp4')
    scorer = NFLAssignmentScorer(labels_video)
    out_deduped = out.groupby(['video_frame','label']).first().reset_index()
    base_video_score = scorer.score(out_deduped)
    
    out_preds = out.drop('label', axis=1).rename(columns={'label_posit':'label'})
    print(out_preds.shape)
    out_preds = out_preds.groupby(['video_frame','label']).first().reset_index()
    print(out_preds.shape)
    deepsort_video_score = scorer.score(out_preds)
    print(f'{base_video_score:0.5f} before --> {deepsort_video_score:0.5f} posit')
    
    
def deepsort_helmets(video_data,video_dir,deepsort_config='deepsort.yaml',plot=False,plot_frames=[]):
    '''func1. Передаем датафрейм из submission_df, полученный из mapping_df. Путь к трейн или тест видео и фрэймы для печати'''
    
    # Setup Deepsort. Инициализируем дипсорт и потом он будет обновляться каждый кадр
    cfg = get_config()
    cfg.merge_from_file(deepsort_config)    
    deepsort = DeepSort(cfg.DEEPSORT.REID_CKPT,max_dist=cfg.DEEPSORT.MAX_DIST,min_confidence=cfg.DEEPSORT.MIN_CONFIDENCE,
                        nms_max_overlap=cfg.DEEPSORT.NMS_MAX_OVERLAP,max_iou_distance=cfg.DEEPSORT.MAX_IOU_DISTANCE,max_age=cfg.DEEPSORT.MAX_AGE,
                        n_init=cfg.DEEPSORT.N_INIT,nn_budget=cfg.DEEPSORT.NN_BUDGET,use_cuda=True)
    
    # Run through frames.
    video_data = video_data.sort_values('frame').reset_index(drop=True) # сортируем кадры по порядку 
    ds = []
    # итерируемся по кадрам. Получаем датасет каждого кадра
    for frame, d in tqdm(video_data.groupby(['frame']), total=video_data['frame'].nunique()):  # list(video_data.groupby(['frame']))[::-1] for descenging order
        d['x'] = (d['left'] + round(d['width'] / 2)) # центр бокса по Х
        d['y'] = (d['top'] + round(d['height'] / 2)) # центр бокса по У
        xywhs = d[['x','y','width','height']].values # значения боксов (центры по X,Y, а также длина и ширина)
        cap = cv2.VideoCapture(f'{video_dir}/{myvideo}.mp4') # обращаемся к видеофайлу
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame-1) # Устанавливаем индекс кадра, который будет извлекаться из видеопотока (индексация начинается от 0)
        success, image = cap.read() # Считываем кадр(картинку) , метод возвращает флаг success (True , False) и image — саму картинку (массив numpy)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # Меняем цветовое пространство на RGB
        confs = np.ones([len(d),]) # создаем массив единиц для confs каждого бокса этого кадра
        clss =  np.zeros([len(d),]) # создаем массив нулей для класса каждого бокса этого кадра
        # в deepsort передаем параметры боксов, confs, классы и картинку. outputs это [x1, y1, x2, y2, track_id, class_id]
        outputs = deepsort.update(xywhs, confs, clss, image) 
        

        # отрисовка промежуточных кадров с боксами из plot_frames
#         if (plot and frame > cfg.DEEPSORT.N_INIT) or (frame in plot_frames): # N_INIT=3
# #             print('To Deepsort:',xywhs, confs, clss, image,sep='\n')
# #             print('Outputs from Deepsort\n',outputs)
#             shown_labels = []
#             for j, (output, conf) in enumerate(zip(outputs, confs)):  # outputs из deepsort-a
#                 # outputs это [x1, y1, x2, y2, track_id, class_id], распарсиваем
#                 bboxes = output[0:4]
#                 id = output[4]
#                 cls = output[5]
#                 c = int(cls)  # integer class
#                 label = f'{id}'
#                 shown_labels.append(label)
#                 color = compute_color_for_id(id)
#                 im = plot_one_box(bboxes, image, label=label, color=color, line_thickness=2)
#             fig, ax = plt.subplots(figsize=(15, 10))
#             video_frame = d['video_frame'].values[0]
#             ax.set_title(f'Deepsort labels: {video_frame}, shown_labels: {shown_labels}')
#             plt.imshow(im)
#             plt.show()

#         outputs из deepsort-a запаковываем в датафрейм, outputs это [x1, y1, x2, y2, track_id, class_id]
        preds_df = pd.DataFrame(outputs, columns=['left','top','right','bottom','deepsort_cluster','class']) #
        if len(preds_df) > 0:
            # TODO Fix this messy merge # This is similar to a left-join except that we match on nearest key. DataFrames must be sorted by the key.
            d = pd.merge_asof(d.sort_values(['left','top']), # исходный датафрейм этого кадра из video_data, по сути submission_df
                              preds_df[['left','top','deepsort_cluster']].sort_values(['left','top']), 
                              on='left', suffixes=('','_deepsort'),direction='nearest') # Whether to search for prior, subsequent, or closest matches 
        ds.append(d) # Складываем покадрово результаты Deepsort-a
    dout = pd.concat(ds)
    return dout

### Posit part (Alex)

In [10]:
# Posit Part (Alex)
def normalize_by_two_dots(xs, ys, pivot_idx, target_idx):
    xs_shifted = xs - xs[pivot_idx]
    ys_shifted = ys - ys[pivot_idx]
#     plt.plot(xs, ys, 'b.')
#     plt.show()
    
    target_x = xs_shifted[target_idx]
    target_y = ys_shifted[target_idx]
#     plt.plot(xs_shifted, ys_shifted, 'b.')
#     plt.show()
    
    if target_x == 0.0:
        if target_y >= 0:
            t = np.pi/2
        else:
            t = 3*np.pi/2
    elif target_y == 0.0:
        if target_x >= 0:
            t = 0
        else:
            t = np.pi
    else:
        if target_x > 0 and target_y > 0:
            quadrant = 1
            tangens = abs(target_y) / abs(target_x)
        elif target_x < 0 and target_y > 0:
            quadrant = 2
            tangens = abs(target_x) / abs(target_y)
        elif target_x < 0 and target_y < 0:
            quadrant = 3
            tangens = abs(target_y) / abs(target_x)
        else:
            quadrant = 4
            tangens = abs(target_x) / abs(target_y)
         
        t = -(np.arctan(tangens) + (quadrant-1)*np.pi/2)
    R = np.array([[np.cos(t), -np.sin(t)],
                  [np.sin(t),  np.cos(t)]])
    u = np.array([xs_shifted, ys_shifted])
    rotated = np.dot(R, u)
#     print(quadrant, tangens, t)
#     plt.plot(rotated[0], rotated[1], 'b.')
#     plt.show()
    rotated_scaled = rotated/rotated[0][target_idx]
    return rotated_scaled

def modern_posit(image_pts, world_pts, focal_length, center):
    nb_points = np.shape(image_pts)[0]

    # centered & scaled pixel coordinates
    centered_image = np.divide(np.subtract(image_pts, center), focal_length)
    ui = centered_image[:, 0]
    vi = centered_image[:, 1]

    # homogeneous world coordinates
    homogeneous_world_pts = np.append(world_pts, np.ones((nb_points, 1)), 1)

    # pseudo inverse
    object_mat = np.linalg.pinv(homogeneous_world_pts)
    #print(object_mat)

    converged = 0
    count = 0
    t_x = 0.0
    t_y = 0.0
    t_z = 0.0
    r1 = 0.0
    r2 = 0.0
    r3 = 0.0
    while converged == 0:
        # POS part of the algorithm
        # rotation vectors
        r1_t = np.matmul(object_mat, ui)
        r2_t = np.matmul(object_mat, vi)
        # 1/t_z1 is norm of r1_t
        t_z1 = 1 / np.linalg.norm(r1_t[0:3])
        # 1/tz_2 is norm of r2_t
        t_z2 = 1 / np.linalg.norm(r2_t[0:3])
        
        # geometric average
        t_z = np.sqrt(t_z1 * t_z2)
        
        r1_n = np.multiply(r1_t, t_z)
        r2_n = np.multiply(r2_t, t_z)
        r1 = r1_n[0:3]
        r2 = r2_n[0:3]
        r3 = np.cross(r1, r2)
        r3_t = np.append(r3, t_z)
        t_x = r1_n[3]
        t_y = r2_n[3]

        # Now update the z/T z or epsilon
        # then ui, vi
        epsilon_i = np.matmul(homogeneous_world_pts, np.divide(r3_t, t_z))
        old_ui = ui
        old_vi = vi
        ui = np.multiply(epsilon_i, centered_image[:, 0])
        vi = np.multiply(epsilon_i, centered_image[:, 1])

        # check for convergence
        delta_ui = ui - old_ui
        delta_vi = vi - old_vi
        delta = np.square(focal_length) * (np.square(np.linalg.norm(delta_ui)) + np.square(np.linalg.norm(delta_vi)))

        converged = 1 if count > 0 and delta < 1 else 0
        count = count + 1

    trans = np.array([t_x, t_y, t_z], np.float64)
    rot = np.array([r1, r2, r3], np.float64)
    return rot, trans

def find_nearest(array, value):
    value = int(value)
    array = np.asarray(array).astype(int)
    idx = (np.abs(array - value)).argmin()
    return array[idx]

def process_with_posit(out, tracking): # Posit начало. На входе результаты дипсорта и трекинг
    PLOT = False
#     Confidence block. Доля самого частого лэйбла от общего количества в заданном кластере дипсорта.
    track_confidence = {}
    for cluster in out['deepsort_cluster'].unique(): # Пробегаемся по кластерам дипсорта
        if not np.isnan(cluster): # Если кластер не None
            label_count = out[out['deepsort_cluster'] == cluster]['label_count_deepsort'].unique()[0] # количество появлений кластера
            num_frames = len(out[out['deepsort_cluster'] == cluster]) # число кадров
            track_confidence[cluster] = label_count/num_frames # как часто кластер появляется на кадре. Словарь Кластер:Частота

    out['track_confidence'] = out['deepsort_cluster'].map(track_confidence) # Добавляем столбец с conf трека

    outs_posit = []
    #for f in range(0, out['frame'].max(), 20):
    errors = []
    #for f in [195]:
    for f in range(1, out['frame'].max()+1): # пробегаемся по кадрам
        out_posit = out[out['frame'] == f].sort_values(['track_confidence'], ascending=False).copy() # результаты дипсорта сортируем по conf
        out_posit_confident = out_posit.head(7).copy() # результаты дипсорта берем топ7 по conf
        
        nearest_frame_tracking = find_nearest(tracking['est_frame'].values, f) # Находим предполагаемый кадр из трека по номеру кадра
        tracking_posit = tracking[tracking['est_frame']==nearest_frame_tracking].copy() # трек предполагаемого кадра
        #out_posit_confident = out_posit[out_posit['track_confidence'] > 0.6].copy()
        
#         Если игрок входит в  Топ7 из лейблов дипсорта по conf 
        tracking_posit_confident = tracking_posit[tracking_posit['player'].isin(out_posit_confident['label_deepsort'].values)].copy()
        tracking_posit_confident['label_posit'] = tracking_posit_confident['player'].copy() # лнйбл это игрок
        
        out_posit_confident['label_posit'] = out_posit_confident['label_deepsort'].copy() # лейбл посита это лейбл дипсорта
        out_posit_confident_duplicates = out_posit_confident['label_deepsort'].value_counts() # считаем количество значений лэйблов дипсорта
        out_posit_confident_duplicates = out_posit_confident_duplicates[out_posit_confident_duplicates > 1] # если больше 1 то складываем в датафрейм
        #print(list(out_posit_confident_duplicates.to_dict().keys())[0])

        dupl_label = "" # заглушка для дублируемого лэйбла
        dupl_label_list = list(out_posit_confident_duplicates.to_dict().keys()) # делаем словарь (спиоск?) из 2+ встречающихся лэйблов дипсорта
        if len(dupl_label_list) > 0: # список длинный
            dupl_label = dupl_label_list[0] # первый лэйбл заполняет заглушку
        #print((out_posit_confident['label_deepsort'] == 'H15').reset_index())
        out_posit_confident_merged = out_posit_confident.merge(tracking_posit_confident, on='label_posit', suffixes = ['','_t'])[['video_frame', 'label_posit', 'x', 'y', 'x_t', 'y_t']].copy()
        #print(out_posit_confident_merged)
        to_find_index = out_posit_confident.reset_index(drop=True).copy()
        #print("to_find_index", to_find_index)
        #print(to_find_index.index[to_find_index['label_deepsort'] == 'H15'].tolist())

        points_2d = np.array([out_posit_confident_merged['x'].values, 720-out_posit_confident_merged['y'].values]).transpose()
        points_3d = np.array([out_posit_confident_merged['x_t'].values, out_posit_confident_merged['y_t'].values]).transpose()
        points_3d = np.append(points_3d, np.zeros((points_3d.shape[0], 1)), axis=1)

        #print("len", len(points_2d))
        num_of_entries = len(points_2d)
        #indexes = random.choices(list(range(points_3d.shape[0])), k=points_3d.shape[0])
        #indexes = random.sample(range(points_3d.shape[0]), points_3d.shape[0]-2)
        dupl_list = to_find_index.index[to_find_index['label_deepsort'] == dupl_label].tolist()

        best_error = float("inf")
        best_matrix = None
        best_projected_points_2d_homogen = None
        if (len(out_posit_confident['label_deepsort']) - len(out_posit_confident['label_deepsort'].unique()) == 1):
            for idx in dupl_list:
                points_2d = np.array([out_posit_confident_merged['x'].values, 720-out_posit_confident_merged['y'].values]).transpose()
                points_3d = np.array([out_posit_confident_merged['x_t'].values, out_posit_confident_merged['y_t'].values]).transpose()
                points_3d = np.append(points_3d, np.zeros((points_3d.shape[0], 1)), axis=1)
                good_indices = list(range(num_of_entries))  # [0, 1, 2, 3, 4, 6]
                good_indices.pop(idx)
                points_2d = np.array([points_2d[i] for i in good_indices])
                points_3d = np.array([points_3d[i] for i in good_indices])
                #rot, trans = modern_posit(points_2d[indexes], points_3d[indexes], 1920, [640, 360])
                rot, trans = modern_posit(points_2d, points_3d, 1920, [640, 360])
                rottrans = np.append(rot, trans.reshape(3,1), axis=1)
                intrin = np.array([
                    [1920, 0, 640],
                    [0, 1920, 360],
                    [0, 0, 1],
                ])
                full_matrix = np.matmul(intrin, rottrans)
                projected_points_2d = np.matmul(full_matrix, np.transpose(np.append(points_3d, np.ones((points_3d.shape[0], 1)), axis=1)))
                projected_points_2d_t = np.transpose(projected_points_2d)
                projected_points_2d_homogen = projected_points_2d_t / projected_points_2d[2].reshape(points_3d.shape[0],1)
                norm_error = np.linalg.norm(points_2d - projected_points_2d_homogen[:,0:2])
                if norm_error < best_error:
                    best_error = norm_error
                    best_matrix = full_matrix
                    best_projected_points_2d_homogen = projected_points_2d_homogen
            norm_error = best_error
            full_matrix = best_matrix
            projected_points_2d_homogen = best_projected_points_2d_homogen
        else:
            rot, trans = modern_posit(points_2d, points_3d, 1920, [640, 360])
            rottrans = np.append(rot, trans.reshape(3,1), axis=1)
            intrin = np.array([
                [1920, 0, 640],
                [0, 1920, 360],
                [0, 0, 1],
            ])
            full_matrix = np.matmul(intrin, rottrans)
            projected_points_2d = np.matmul(full_matrix, np.transpose(np.append(points_3d, np.ones((points_3d.shape[0], 1)), axis=1)))
            projected_points_2d_t = np.transpose(projected_points_2d)
            projected_points_2d_homogen = projected_points_2d_t / projected_points_2d[2].reshape(points_3d.shape[0],1)
            norm_error = np.linalg.norm(points_2d - projected_points_2d_homogen[:,0:2])

        if norm_error == float("inf"):
            norm_error = 0.0
        #print(f, 'norm:', norm_error, f,nearest_frame_tracking, len(out_posit_confident), len(out_posit_confident['label_deepsort']) - len(out_posit_confident['label_deepsort'].unique()))
        errors.append(norm_error)

        if PLOT:
            plt.plot(points_2d[:, 0], points_2d[:, 1], 'rx')
            plt.plot(projected_points_2d_homogen[:, 0], projected_points_2d_homogen[:, 1], 'b.')
            plt.axis('scaled')
            plt.show()

            x = projected_points_2d_homogen[:, 0]
            y = projected_points_2d_homogen[:, 1]
            labels = out_posit_confident_merged['label_posit'].values
            labels_posit = out_posit_confident['label_posit'].values
            fig, ax = plt.subplots()
            fig.set_size_inches((15, 15))
            ax.scatter(points_2d[:, 0], points_2d[:, 1], marker='x', color='r')
            ax.scatter(x, y, marker='.', color='b')
            for i in range(len(x)):
                ax.annotate(labels[i], (x[i], y[i]), color='b')

            for i in range(points_2d.shape[0]):
                ax.annotate(labels_posit[i], (points_2d[:, 0][i], points_2d[:, 1][i]), color='r')


        points_2d_full = np.array([out_posit['x'].values, 720-out_posit['y'].values]).transpose()
        points_3d_full = np.array([tracking_posit['x'].values, tracking_posit['y'].values]).transpose()
        points_3d_full = np.append(points_3d_full, np.zeros((points_3d_full.shape[0], 1)), axis=1)

        projected_points_2d_full = np.matmul(full_matrix, np.transpose(np.append(points_3d_full, np.ones((points_3d_full.shape[0], 1)), axis=1)))
        projected_points_2d_t_full = np.transpose(projected_points_2d_full)
        projected_points_2d_homogen_full = projected_points_2d_t_full / projected_points_2d_full[2].reshape(points_3d_full.shape[0],1)
        tracking_posit["x_pr"] = projected_points_2d_homogen_full[:, 0]
        tracking_posit["y_pr"] = projected_points_2d_homogen_full[:, 1]

        if PLOT:
            figure(figsize=(10,10))
            plt.plot(points_2d_full[:, 0], points_2d_full[:, 1], 'rx')
            plt.plot(projected_points_2d_homogen_full[:, 0], projected_points_2d_homogen_full[:, 1], 'b.')
            plt.axis('scaled')
            plt.show()

            x = projected_points_2d_homogen_full[:, 0]
            y = projected_points_2d_homogen_full[:, 1]
            labels = tracking_posit['player'].values
            labels_posit = out_posit['label'].values
            fig, ax = plt.subplots()
            ax.axis('equal')

            fig.set_size_inches((30, 30))
            ax.scatter(points_2d_full[:, 0], points_2d_full[:, 1], marker='x', color='r')
            ax.scatter(x, y, marker='.', color='b')
            for i in range(len(x)):
                ax.annotate(labels[i], (x[i], y[i]), color='b')

            for i in range(len(labels_posit)):
                ax.annotate(labels_posit[i], (points_2d_full[:, 0][i], points_2d_full[:, 1][i]), color='r')


        out_posit['label_posit'] = out_posit['label_deepsort'].values.copy()
        out_posit['y_im'] = (720 - out_posit['y']).copy()
        indexes_to_all_distances ={}
        for index, row in out_posit.iterrows():
            all_distances = np.sqrt(np.square(tracking_posit['x_pr'].values - row['x']) + np.square(tracking_posit['y_pr'].values - row['y_im']))
            all_labels = tracking_posit['player'].values 
            indexes_to_all_distances[index] = sorted(list(zip(all_labels, all_distances)), key=lambda tup: tup[1])


        all_results = {}
        expected_num_of_results = len(indexes_to_all_distances)
        for _ in range(expected_num_of_results):
            min_element = None
            min_dist = float('inf')
            min_label = None
            for k, v in indexes_to_all_distances.items():
                if v[0][1] < min_dist:
                    min_dist = v[0][1]
                    min_element = k
                    min_label = v[0][0]
            if min_element:
                all_results[min_element] = min_label
            indexes_to_all_distances.pop(min_element, None)
            for k, v in indexes_to_all_distances.items():
                indexes_to_all_distances[k] = [t for t in v if t[0] != min_label]

        for k, v in all_results.items():
            out_posit.loc[k,'label_posit'] = v

        outs_posit.append(out_posit.copy())

    submission_posit = pd.concat(outs_posit).copy()
    return submission_posit

In [11]:
# Add video and frame columns to submission.
submission_df['video'] = submission_df['video_frame'].str.split('_').str[:3].str.join('_')
submission_df['frame'] = submission_df['video_frame'].str.split('_').str[-1].astype('int')

if debug:
    video_dir = 'input/nfl-health-and-safety-helmet-assignment/train/'
else:
    video_dir = 'input/nfl-health-and-safety-helmet-assignment/test/'

submission_df_small = submission_df.loc[submission_df['frame']<51].copy()
submission_df_small.frame.unique()

array([ 1, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,  2, 20, 21, 22, 23, 24,
       25, 26, 27, 28, 29,  3, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39,  4,
       40, 41, 42, 43, 44, 45, 46, 47, 48, 49,  5, 50,  6,  7,  8,  9])

In [10]:
# ОПТИМИЗАЦИЯ
#     track_confidence = {}
#     for cluster in out['deepsort_cluster'].unique(): # Пробегаемся по кластерам дипсорта
#         if not np.isnan(cluster): # Если кластер не None, то есть после пропускаемых дипсортом кадров
#             label_count = out[out['deepsort_cluster'] == cluster]['label_count_deepsort'].unique()[0] # количество появлений топ кластера
#             num_frames = len(out[out['deepsort_cluster'] == cluster]) # число кадров
# #             print(cluster, 'label_count',label_count,'num_frames',num_frames)
#             track_confidence[cluster] = label_count/num_frames # как часто кластер появляется на кадре. Словарь Кластер:Частота
# #     print('track_confidence\n',track_confidence)
# replace to this 
# count_clusters = out.groupby(['deepsort_cluster','label'], as_index=False).count()
# track_confidence = (count_clusters.groupby(['deepsort_cluster']).max().video_frame / count_clusters.groupby(['deepsort_cluster']).sum().video_frame).to_dict()

In [13]:
# Loop through test videos and apply. If in debug mode show the score change.
outs = []
outs_posit = []
for myvideo, video_data in tqdm(submission_df_small.groupby('video'), total=submission_df['video'].nunique()):
    print(f'==== {myvideo} ====')
    if debug:
        # Plot deepsort labels when in debug mode.
        out = deepsort_helmets(video_data, video_dir, plot_frames=range(3,100,10)) #i for i in range(0,1000,20)] + [379, 381]
#         print(out)
    else:
        out = deepsort_helmets(video_data, video_dir)      
    # Находим лэйбл, который в наибольшем количестве кадров присутствовал с определенным кластером. Добавляются label_deepsort (ИД игрока) label_count_deepsort (число)
    out = add_deepsort_label_col(out) 
    outs.append(out) # итоговые датафреймы складываются в список
    #####
    tracking_gp = tracking[tracking['game_play'] == '_'.join((myvideo.split('_'))[:-1])].copy() # просто трек текущего видео
    out_posit = process_with_posit(out, tracking_gp) # Posit
    outs_posit.append(out_posit)
    #####
    if debug:
        score_vs_deepsort(myvideo, out, labels)
        score_vs_posit(myvideo, out_posit, labels) #
        
submission_deepsort = pd.concat(outs).copy()
submission_posit = pd.concat(outs_posit).copy()
# ==== 57783_003374_Endzone ====
# 0.27200 before --> 0.63371 deepsort --> 0.67545 posit
# ==== 57783_003374_Sideline ====
# 0.29314 before --> 0.32986 deepsort --> 0.60836 posit

# ==== 57783_003374_Endzone ==== 50 frames
# 0.09549 before --> 0.30871 deepsort --> 0.31109 posit
# ==== 57783_003374_Sideline ==== 50 frames
# 0.09459 before --> 0.05549 deepsort --> 0.01022 posit

  0%|          | 0/2 [00:00<?, ?it/s]

==== 57783_003374_Endzone ====


100%|██████████| 50/50 [00:53<00:00,  1.07s/it]
/Users/apple/opt/anaconda3/envs/Py37/lib/python3.7/site-packages/ipykernel_launcher.py:77: RuntimeWarning: divide by zero encountered in double_scalars
/Users/apple/opt/anaconda3/envs/Py37/lib/python3.7/site-packages/ipykernel_launcher.py:79: RuntimeWarning: divide by zero encountered in double_scalars
/Users/apple/opt/anaconda3/envs/Py37/lib/python3.7/site-packages/ipykernel_launcher.py:84: RuntimeWarning: invalid value encountered in multiply
/Users/apple/opt/anaconda3/envs/Py37/lib/python3.7/site-packages/ipykernel_launcher.py:85: RuntimeWarning: invalid value encountered in multiply
/Users/apple/opt/anaconda3/envs/Py37/lib/python3.7/site-packages/ipykernel_launcher.py:95: RuntimeWarning: invalid value encountered in true_divide
/Users/apple/opt/anaconda3/envs/Py37/lib/python3.7/site-packages/ipykernel_launcher.py:77: RuntimeWarning: divide by zero encountered in double_scalars
/Users/apple/opt/anaconda3/envs/Py37/lib/python3.7/site-pa

(827, 14)
(787, 14)
0.09549 before --> 0.30871 deepsort
(827, 16)
(795, 16)


 50%|█████     | 1/2 [00:54<00:54, 54.90s/it]

0.09549 before --> 0.31109 posit
==== 57783_003374_Sideline ====


100%|██████████| 50/50 [01:09<00:00,  1.39s/it]
/Users/apple/opt/anaconda3/envs/Py37/lib/python3.7/site-packages/ipykernel_launcher.py:77: RuntimeWarning: divide by zero encountered in double_scalars
/Users/apple/opt/anaconda3/envs/Py37/lib/python3.7/site-packages/ipykernel_launcher.py:79: RuntimeWarning: divide by zero encountered in double_scalars
/Users/apple/opt/anaconda3/envs/Py37/lib/python3.7/site-packages/ipykernel_launcher.py:84: RuntimeWarning: invalid value encountered in multiply
/Users/apple/opt/anaconda3/envs/Py37/lib/python3.7/site-packages/ipykernel_launcher.py:85: RuntimeWarning: invalid value encountered in multiply
/Users/apple/opt/anaconda3/envs/Py37/lib/python3.7/site-packages/ipykernel_launcher.py:95: RuntimeWarning: invalid value encountered in true_divide
/Users/apple/opt/anaconda3/envs/Py37/lib/python3.7/site-packages/ipykernel_launcher.py:77: RuntimeWarning: divide by zero encountered in double_scalars
/Users/apple/opt/anaconda3/envs/Py37/lib/python3.7/site-pa

(1085, 14)
(857, 14)
0.09459 before --> 0.05549 deepsort
(1085, 16)
(1041, 16)


100%|██████████| 2/2 [02:05<00:00, 62.80s/it]

0.09459 before --> 0.01022 posit


In [20]:
out.to_csv('tstr/out.csv')
tracking_gp.to_csv('tstr/tracking_gp.csv')

In [16]:
# %%time
def process_with_posit(out, tracking): # Posit начало. 
    '''
    IN: результаты дипсорта с самым частым лэйблом и его значением, а также трекинг
    
    '''
    # print('out\n',out)
    # print('tracking\n',tracking)
    PLOT = False
    
    track_confidence = {}
    for cluster in out['deepsort_cluster'].unique(): # Пробегаемся по кластерам дипсорта
        if not np.isnan(cluster): # Если кластер не None, то есть после пропускаемых дипсортом кадров
            label_count = out[out['deepsort_cluster'] == cluster]['label_count_deepsort'].unique()[0] # количество появлений топ кластера
            num_frames = len(out[out['deepsort_cluster'] == cluster]) # число кадров
            # print(cluster, 'label_count',label_count,'num_frames',num_frames)
            track_confidence[cluster] = label_count/num_frames # как часто кластер появляется на кадре. Словарь Кластер:Частота


    out['track_confidence'] = out['deepsort_cluster'].map(track_confidence) # Добавляем столбец с conf трека
    # print(out,'\n')
    outs_posit = []
    #for f in range(0, out['frame'].max(), 20):
    errors = []
    #for f in [195]:
    for f in [4]:#(1, out['frame'].max()+1): # пробегаемся по кадрам                
        print('\t\t\t',f)
        # результаты дипсорта сортируем по conf кластера из цикла выше и берем топ7 по conf
        out_posit = out[out['frame'] == f].sort_values(['track_confidence'], ascending=False).copy() 
        out_posit_confident = out_posit.head(7).copy() 
        print('======out_posit результаты дипсорта сортируем по conf кластера\n',out_posit)
        print('======out_posit_confident берем топ7 по conf\n',out_posit_confident)
        
        # находим в треке примерный кадр и фиксируем его трек
        nearest_frame_tracking = find_nearest(tracking['est_frame'].values, f) # int номер кадра
        tracking_posit = tracking[tracking['est_frame']==nearest_frame_tracking].copy() # трек этого кадра
        #out_posit_confident = out_posit[out_posit['track_confidence'] > 0.6].copy()

        print('======nearest_frame_tracking находим в треке примерный кадр\n',nearest_frame_tracking)
        print('======tracking_posit фиксируем его трек\n',tracking_posit)
        
        # Если игрок входит в  Топ7 из кластеров дипсорта по conf 
        tracking_posit_confident = tracking_posit[tracking_posit['player'].isin(out_posit_confident['label_deepsort'].values)].copy()
        print('======tracking_posit_confident Если игрок входит в  Топ7 из кластеров дипсорта по conf \n',tracking_posit_confident)
        tracking_posit_confident['label_posit'] = tracking_posit_confident['player'].copy() # лейбл это игрок
        print('======tracking_posit_confident лейбл это игрок\n',tracking_posit_confident)
        
        out_posit_confident['label_posit'] = out_posit_confident['label_deepsort'].copy() # лейбл посита делаем из лейбла дипсорта
        print('======out_posit_confident лейбл посита делаем из лейбла дипсорта\n',out_posit_confident)
        out_posit_confident_duplicates = out_posit_confident['label_deepsort'].value_counts() # считаем количество значений лэйблов дипсорта
        print('======out_posit_confident_duplicates считаем количество значений лэйблов дипсорта\n',out_posit_confident_duplicates)
        out_posit_confident_duplicates = out_posit_confident_duplicates[out_posit_confident_duplicates > 1] # если больше 1 то складываем в датафрейм
        print('======out_posit_confident_duplicates если больше 1 то складываем в датафрейм\n',out_posit_confident_duplicates)
        #print(list(out_posit_confident_duplicates.to_dict().keys())[0])

        dupl_label = "" # заглушка для дублируемого лэйбла
        dupl_label_list = list(out_posit_confident_duplicates.to_dict().keys()) # делаем словарь (спиоск?) из 2+ встречающихся лэйблов дипсорта
        print('======dupl_label_list # делаем словарь (спиоск?) из 2+ встречающихся лэйблов дипсорта\n',dupl_label_list)
        if len(dupl_label_list) > 0: # список длинный
            dupl_label = dupl_label_list[0] # первый лэйбл заполняет заглушку
        print('======dupl_label\n',dupl_label)
        print('======dupl_label_list[0]\n',dupl_label_list)
        #print((out_posit_confident['label_deepsort'] == 'H15').reset_index())
        out_posit_confident_merged = out_posit_confident.merge(tracking_posit_confident, on='label_posit', suffixes = ['','_t'])[['video_frame', 'label_posit', 'x', 'y', 'x_t', 'y_t']].copy()
        print('======out_posit_confident_merged\n',out_posit_confident_merged)
        #print(out_posit_confident_merged)
        to_find_index = out_posit_confident.reset_index(drop=True).copy()
        print('======to_find_index\n',to_find_index)
        #print("to_find_index", to_find_index)
        #print(to_find_index.index[to_find_index['label_deepsort'] == 'H15'].tolist())

        points_2d = np.array([out_posit_confident_merged['x'].values, 720-out_posit_confident_merged['y'].values]).transpose()
        print('======points_2d\n',points_2d)
        points_3d = np.array([out_posit_confident_merged['x_t'].values, out_posit_confident_merged['y_t'].values]).transpose()
        print('======points_3d\n',points_3d)
        points_3d = np.append(points_3d, np.zeros((points_3d.shape[0], 1)), axis=1)
        print('======points_3d\n',points_3d)

        #print("len", len(points_2d))
        num_of_entries = len(points_2d)
        print('======num_of_entries\n',num_of_entries)
        #indexes = random.choices(list(range(points_3d.shape[0])), k=points_3d.shape[0])
        #indexes = random.sample(range(points_3d.shape[0]), points_3d.shape[0]-2)
        dupl_list = to_find_index.index[to_find_index['label_deepsort'] == dupl_label].tolist()
        print('======dupl_list\n',dupl_list)
        
        
        
        best_error = float("inf")
        best_matrix = None
        best_projected_points_2d_homogen = None
        if (len(out_posit_confident['label_deepsort']) - len(out_posit_confident['label_deepsort'].unique()) == 1):
            for idx in dupl_list:
                points_2d = np.array([out_posit_confident_merged['x'].values, 720-out_posit_confident_merged['y'].values]).transpose()
                points_3d = np.array([out_posit_confident_merged['x_t'].values, out_posit_confident_merged['y_t'].values]).transpose()
                points_3d = np.append(points_3d, np.zeros((points_3d.shape[0], 1)), axis=1)
                good_indices = list(range(num_of_entries))  # [0, 1, 2, 3, 4, 6]
                good_indices.pop(idx)
                points_2d = np.array([points_2d[i] for i in good_indices])
                points_3d = np.array([points_3d[i] for i in good_indices])
                #rot, trans = modern_posit(points_2d[indexes], points_3d[indexes], 1920, [640, 360])
                rot, trans = modern_posit(points_2d, points_3d, 1920, [640, 360])
                rottrans = np.append(rot, trans.reshape(3,1), axis=1)
                intrin = np.array([
                    [1920, 0, 640],
                    [0, 1920, 360],
                    [0, 0, 1],
                ])
                full_matrix = np.matmul(intrin, rottrans)
                projected_points_2d = np.matmul(full_matrix, np.transpose(np.append(points_3d, np.ones((points_3d.shape[0], 1)), axis=1)))
                projected_points_2d_t = np.transpose(projected_points_2d)
                projected_points_2d_homogen = projected_points_2d_t / projected_points_2d[2].reshape(points_3d.shape[0],1)
                norm_error = np.linalg.norm(points_2d - projected_points_2d_homogen[:,0:2])
                if norm_error < best_error:
                    best_error = norm_error
                    best_matrix = full_matrix
                    best_projected_points_2d_homogen = projected_points_2d_homogen
            norm_error = best_error
            full_matrix = best_matrix
            projected_points_2d_homogen = best_projected_points_2d_homogen
        else:
            rot, trans = modern_posit(points_2d, points_3d, 1920, [640, 360])
            rottrans = np.append(rot, trans.reshape(3,1), axis=1)
            intrin = np.array([
                [1920, 0, 640],
                [0, 1920, 360],
                [0, 0, 1],
            ])
            full_matrix = np.matmul(intrin, rottrans)
            projected_points_2d = np.matmul(full_matrix, np.transpose(np.append(points_3d, np.ones((points_3d.shape[0], 1)), axis=1)))
            projected_points_2d_t = np.transpose(projected_points_2d)
            projected_points_2d_homogen = projected_points_2d_t / projected_points_2d[2].reshape(points_3d.shape[0],1)
            norm_error = np.linalg.norm(points_2d - projected_points_2d_homogen[:,0:2])

        if norm_error == float("inf"):
            norm_error = 0.0
        #print(f, 'norm:', norm_error, f,nearest_frame_tracking, len(out_posit_confident), len(out_posit_confident['label_deepsort']) - len(out_posit_confident['label_deepsort'].unique()))
        errors.append(norm_error)

        if PLOT:
            plt.plot(points_2d[:, 0], points_2d[:, 1], 'rx')
            plt.plot(projected_points_2d_homogen[:, 0], projected_points_2d_homogen[:, 1], 'b.')
            plt.axis('scaled')
            plt.show()

            x = projected_points_2d_homogen[:, 0]
            y = projected_points_2d_homogen[:, 1]
            labels = out_posit_confident_merged['label_posit'].values
            labels_posit = out_posit_confident['label_posit'].values
            fig, ax = plt.subplots()
            fig.set_size_inches((15, 15))
            ax.scatter(points_2d[:, 0], points_2d[:, 1], marker='x', color='r')
            ax.scatter(x, y, marker='.', color='b')
            for i in range(len(x)):
                ax.annotate(labels[i], (x[i], y[i]), color='b')

            for i in range(points_2d.shape[0]):
                ax.annotate(labels_posit[i], (points_2d[:, 0][i], points_2d[:, 1][i]), color='r')


        points_2d_full = np.array([out_posit['x'].values, 720-out_posit['y'].values]).transpose()
        points_3d_full = np.array([tracking_posit['x'].values, tracking_posit['y'].values]).transpose()
        points_3d_full = np.append(points_3d_full, np.zeros((points_3d_full.shape[0], 1)), axis=1)

        projected_points_2d_full = np.matmul(full_matrix, np.transpose(np.append(points_3d_full, np.ones((points_3d_full.shape[0], 1)), axis=1)))
        projected_points_2d_t_full = np.transpose(projected_points_2d_full)
        projected_points_2d_homogen_full = projected_points_2d_t_full / projected_points_2d_full[2].reshape(points_3d_full.shape[0],1)
        tracking_posit["x_pr"] = projected_points_2d_homogen_full[:, 0]
        tracking_posit["y_pr"] = projected_points_2d_homogen_full[:, 1]

        if PLOT:
            figure(figsize=(10,10))
            plt.plot(points_2d_full[:, 0], points_2d_full[:, 1], 'rx')
            plt.plot(projected_points_2d_homogen_full[:, 0], projected_points_2d_homogen_full[:, 1], 'b.')
            plt.axis('scaled')
            plt.show()

            x = projected_points_2d_homogen_full[:, 0]
            y = projected_points_2d_homogen_full[:, 1]
            labels = tracking_posit['player'].values
            labels_posit = out_posit['label'].values
            fig, ax = plt.subplots()
            ax.axis('equal')

            fig.set_size_inches((30, 30))
            ax.scatter(points_2d_full[:, 0], points_2d_full[:, 1], marker='x', color='r')
            ax.scatter(x, y, marker='.', color='b')
            for i in range(len(x)):
                ax.annotate(labels[i], (x[i], y[i]), color='b')

            for i in range(len(labels_posit)):
                ax.annotate(labels_posit[i], (points_2d_full[:, 0][i], points_2d_full[:, 1][i]), color='r')


        out_posit['label_posit'] = out_posit['label_deepsort'].values.copy()
        out_posit['y_im'] = (720 - out_posit['y']).copy()
        indexes_to_all_distances ={}
        for index, row in out_posit.iterrows():
            all_distances = np.sqrt(np.square(tracking_posit['x_pr'].values - row['x']) + np.square(tracking_posit['y_pr'].values - row['y_im']))
            all_labels = tracking_posit['player'].values 
            indexes_to_all_distances[index] = sorted(list(zip(all_labels, all_distances)), key=lambda tup: tup[1])


        all_results = {}
        expected_num_of_results = len(indexes_to_all_distances)
        for _ in range(expected_num_of_results):
            min_element = None
            min_dist = float('inf')
            min_label = None
            for k, v in indexes_to_all_distances.items():
                if v[0][1] < min_dist:
                    min_dist = v[0][1]
                    min_element = k
                    min_label = v[0][0]
            if min_element:
                all_results[min_element] = min_label
            indexes_to_all_distances.pop(min_element, None)
            for k, v in indexes_to_all_distances.items():
                indexes_to_all_distances[k] = [t for t in v if t[0] != min_label]

        for k, v in all_results.items():
            out_posit.loc[k,'label_posit'] = v

        outs_posit.append(out_posit.copy())

    submission_posit = pd.concat(outs_posit).copy()
    return submission_posit

process_with_posit(out, tracking_gp)

			 4
======out_posit результаты дипсорта сортируем по conf кластера
                 video_frame  left  width  top  height label  \
0   57783_003374_Sideline_4   174     13  259      15   V25   
21  57783_003374_Sideline_4   972     16  386      16   H25   
1   57783_003374_Sideline_4   496     16  358      16   V55   
8   57783_003374_Sideline_4   686     12  361      16   V99   
4   57783_003374_Sideline_4   540     14  301      17   V31   
7   57783_003374_Sideline_4   664     13  420      16   V96   
3   57783_003374_Sideline_4   525     15  498      16   V54   
19  57783_003374_Sideline_4   808     14  183      15   H14   
18  57783_003374_Sideline_4   760     12   60      14   H72   
2   57783_003374_Sideline_4   499     15  414      15   V38   
20  57783_003374_Sideline_4   836     12   54      14   H15   
5   57783_003374_Sideline_4   642     14  474      15   V57   
6   57783_003374_Sideline_4   643     13  188      16   V98   
17  57783_003374_Sideline_4   735     13  317   

,video_frame,left,width,top,height,label,video,frame,x,y,top_deepsort,deepsort_cluster,label_deepsort,label_count_deepsort,track_confidence,label_posit,y_im
0,57783_003374_Sideline_4,174,13,259,15,V25,57783_003374_Sideline,4,180.0,267.0,259.0,1.0,V25,48.0,1.000000,V25,453.0
21,57783_003374_Sideline_4,972,16,386,16,H25,57783_003374_Sideline,4,980.0,394.0,386.0,12.0,H25,48.0,1.000000,H25,326.0
1,57783_003374_Sideline_4,496,16,358,16,V55,57783_003374_Sideline,4,504.0,366.0,358.0,21.0,V55,35.0,0.897436,V55,354.0
8,57783_003374_Sideline_4,686,12,361,16,V99,57783_003374_Sideline,4,692.0,369.0,361.0,14.0,V99,23.0,0.851852,V96,351.0
4,57783_003374_Sideline_4,540,14,301,17,V31,57783_003374_Sideline,4,547.0,309.0,300.0,18.0,V31,39.0,0.829787,V31,411.0
7,57783_003374_Sideline_4,664,13,420,16,V96,57783_003374_Sideline,4,670.0,428.0,420.0,15.0,V96,38.0,0.826087,V99,292.0
3,57783_003374_Sideline_4,525,15,498,16,V54,57783_003374_Sideline,4,533.0,506.0,498.0,19.0,V54,40.0,0.816327,V38,214.0
19,57783_003374_Sideline_4,808,14,183,15,H14,57783_003374_Sideline,4,815.0,191.0,183.0,3.0,H14,37.0,0.787234,H12,529.0
18,57783_003374_Sideline_4,760,12,60,14,H72,57783_003374_Sideline,4,766.0,67.0,59.0,4.0,H72,36.0,0.782609,H14,653.0
2,57783_003374_Sideline_4,499,15,414,15,V38,57783_003374_Sideline,4,507.0,422.0,414.0,20.0,V38,44.0,0.771930,V54,298.0


In [ ]:
cap = cv2.VideoCapture(f'input/nfl-health-and-safety-helmet-assignment/train/57783_003374_Endzone.mp4') # обращаемся к видеофайлу
frame = 10
cap.set(cv2.CAP_PROP_POS_FRAMES, frame-1) # Устанавливаем индекс кадра, который будет извлекаться из видеопотока (индексация начинается от 0)
success, image = cap.read() # Считываем кадр(картинку) , метод возвращает флаг success (True , False) и image — саму картинку (массив numpy)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # Меняем цветовое пространство на RGB
# shown_labels = []
# for j, (output, conf) in enumerate(zip(outputs, confs)):  # outputs из deepsort-a
#     # outputs это [x1, y1, x2, y2, track_id, class_id], распарсиваем
#     bboxes = output[0:4]
#     id = output[4]
#     cls = output[5]
#     c = int(cls)  # integer class
#     label = f'{id}'
#     shown_labels.append(label)
#     color = compute_color_for_id(id)
#     im = plot_one_box(bboxes, image, label=label, color=color, line_thickness=2)
# fig, ax = plt.subplots(figsize=(15, 10))
# video_frame = d['video_frame'].values[0]
# ax.set_title(f'Deepsort labels: {video_frame}, shown_labels: {shown_labels}')
# plt.imshow(im)
# plt.show()
im = plot_one_box(submission_deepsort[submission_deepsort.frame == frame], image, label=label, color=color, line_thickness=2)
plt.imshow(image)

# Check Submission & Save
Finally we will create a submission file and check that it passes the submission requirements.
The steps are:
1. Drop the `label` and replace with `label_deepsort` predictions.
2. Remove any duplicate labels within a single video/frame. This is required to meet the submission requirements.
3. Save the results.

In [ ]:
# Deepsort Submission
ss = pd.read_csv('input/nfl-health-and-safety-helmet-assignment/sample_submission.csv')
# Final Checks
submission_deepsort['label_deepsort'] = submission_deepsort['label_deepsort'].fillna(submission_deepsort['label'])
submission_posit['label_posit'] = submission_posit['label_posit'].fillna(submission_posit['label'])

submission_deepsort = submission_deepsort.drop('label', axis=1).rename(columns={'label_deepsort':'label'})[ss.columns]
submission_posit = submission_posit.drop('label', axis=1).rename(columns={'label_posit':'label'})[ss.columns]
# Drop duplicate labels
submission_deepsort = submission_deepsort.loc[~submission_deepsort[['video_frame','label']].duplicated()]
submission_posit = submission_posit.loc[~submission_posit[['video_frame','label']].duplicated()]

check_submission(submission_deepsort)
check_submission(submission_posit)

# submission_deepsort.to_csv('submission.csv', index=False)
submission_posit.to_csv('submission.csv', index=False)

### def dots():

In [ ]:
def dots(game_play, baseline_boxes, gt_labels, tracking, frame, home_color="red", away_color="white"): # номер игры, боксы, лэйблы, трекинг и номер кадра

    baseline_boxes['game_play'] = baseline_boxes['video_frame'].map(lambda x: '_'.join(str(x).split('_')[:2]))
    baseline_boxes['frame'] = baseline_boxes['video_frame'].map(lambda x: int(str(x).split('_')[-1]))
    baseline_boxes['zone'] = baseline_boxes['video_frame'].map(lambda x: str(x).split('_')[-2])
    
    ez = "Endzone"
    sd = "Sideline"
    game_play_boxes_endzone = baseline_boxes.query("game_play == @game_play and frame == @frame and zone == @ez").reset_index(drop=True)
    game_play_boxes_sideline = baseline_boxes.query("game_play == @game_play and frame == @frame and zone == @sd").reset_index(drop=True)
    
    cap = cv2.VideoCapture(f"input/nfl-health-and-safety-helmet-assignment/train/{game_play}_Endzone.mp4")
    cap.set(1, frame-1)
    _, image_endzone = cap.read()
    cap = cv2.VideoCapture(f"input/nfl-health-and-safety-helmet-assignment/train/{game_play}_Sideline.mp4")
    cap.set(1, frame-1)
    _, image_sideline = cap.read()
    fig, axs = plt.subplots(2, 2, figsize=(30, 15))
    
    game_play_tracking = tracking.query("game_play == @game_play").reset_index(drop=True)
    closest_tracking_frame = game_play_tracking.iloc[(game_play_tracking['est_frame']-frame).abs().argsort()[0]]['est_frame']
    game_play_tracking = game_play_tracking.query("est_frame == @closest_tracking_frame").reset_index(drop=True)
    
    image_endzone_black = image_endzone*0
    image_sideline_black = image_sideline*0
    
    color = (255, 255, 255)
    
    # Now, add the boxes
    for box in game_play_boxes_endzone.itertuples(index=False):
        cv2.rectangle(
            image_endzone,
            (box.left, box.top),
            (box.left + box.width, box.top + box.height),
            color,
            thickness=1,
        )
        cv2.putText(
            image_endzone,
            f"{box.conf:0.2}",
            (box.left, max(0, box.top - 5)),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.5,
            color,
            thickness=1,
        )
        cv2.circle(image_endzone_black, (box.left + box.width//2 ,box.top + box.height//2), radius=5, color=color, thickness=-1)
    
    for box in game_play_boxes_sideline.itertuples(index=False):
        cv2.rectangle(
            image_sideline,
            (box.left, box.top),
            (box.left + box.width, box.top + box.height),
            color,
            thickness=1,
        )
        cv2.putText(
            image_sideline,
            f"{box.conf:0.2}",
            (box.left, max(0, box.top - 5)),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.5,
            color,
            thickness=1,
        )
        cv2.circle(image_sideline_black, (box.left + box.width//2 ,box.top + box.height//2), radius=5, color=color, thickness=-1)
    
    axs[0, 0].imshow(cv2.cvtColor(image_sideline, cv2.COLOR_BGR2RGB))
    axs[0, 0].axis("off")
    axs[0, 0].set_title(f"{game_play}_Endzone_{frame}", fontsize=20)
    axs[0, 1].imshow(cv2.cvtColor(image_endzone, cv2.COLOR_BGR2RGB))
    axs[0, 1].set_title(f"{game_play}_Sideline_{frame}", fontsize=20)
    axs[0, 1].axis("off")
    axs[1, 0].imshow(cv2.cvtColor(image_sideline_black, cv2.COLOR_BGR2RGB))
    axs[1, 0].axis("off")
    axs[1, 0].set_title(f"{game_play}_Endzone_{frame}", fontsize=20)
    axs[1, 1].imshow(cv2.cvtColor(image_endzone_black, cv2.COLOR_BGR2RGB))
    axs[1, 1].set_title(f"{game_play}_Sideline_{frame}", fontsize=20)
    axs[1, 1].axis("off")
    plt.tight_layout()
    
    ax = create_football_field()
    team_to_color = {"Home" : "red", "Away" : "white"}
    for team, d in game_play_tracking.groupby("team"):
        ax.scatter(d["x"], d["y"], label=team, s=65, lw=1, edgecolors="black", zorder=5, color=team_to_color[team])
    ax.legend().remove()
    ax.set_title(f"Tracking data for {game_play}: at {closest_tracking_frame}", fontsize=15)
    plt.show()

    return game_play_boxes_endzone, game_play_boxes_sideline, game_play_tracking

In [ ]:
def score_on_video(myvideo, preds, labels):
    myvideo_mp4 = myvideo + '.mp4'
    labels_video = labels.query('video == @myvideo_mp4')
    scorer = NFLAssignmentScorer(labels_video)
    video_score = scorer.score(preds)
    print(f'score: {video_score:0.5f}')
    
def score_on_all_videos(preds, labels):
    scorer = NFLAssignmentScorer(labels)
    video_score = scorer.score(preds)
    print(f'score: {video_score:0.5f}')

# Visualizations

# Display video showing predictions

Lastly, if we want to review our predictions we can create a video to review the predictions using the `video_with_predictions` function from the `helmet_assignment` helper package.

In [ ]:
from helmet_assignment.video import video_with_predictions
from IPython.display import Video, display

if debug:
    submission_deepsort['video'] = submission_deepsort['video_frame'].str.split('_').str[:3].str.join('_') + '.mp4'
    debug_videos = submission_deepsort['video'].unique()
    debug_labels = labels.query('video in @debug_videos')
    scorer = NFLAssignmentScorer(debug_labels)
    scorer.score(submission_deepsort)
    
    # Create video showing predictions for one of the videos.
    video_out = video_with_predictions(f'input/nfl-health-and-safety-helmet-assignment/train/{debug_videos[0]}',
                                       scorer.sub_labels)
    
    frac = 0.60 # scaling factor for display
    display(Video(data=video_out,embed=True,height=int(720*frac),width=int(1280*frac)))

In [ ]:
# Matplotlib Field
import matplotlib.patches as patches
import matplotlib.pylab as plt

def create_football_field(
    linenumbers=True,
    endzones=True,
    highlight_line=False,
    highlight_line_number=50,
    highlighted_name="Line of Scrimmage",
    fifty_is_los=False,
    figsize=(12, 6.33),
    field_color="lightgreen",
    ez_color='forestgreen',
    ax=None,
):
    """
    Function that plots the football field for viewing plays.
    Allows for showing or hiding endzones.
    """
    rect = patches.Rectangle(
        (0, 0),
        120,
        53.3,
        linewidth=0.1,
        edgecolor="r",
        facecolor=field_color,
        zorder=0,
    )

    if ax is None:
        fig, ax = plt.subplots(1, figsize=figsize)
    ax.add_patch(rect)

    plt.plot([10, 10, 10, 20, 20, 30, 30, 40, 40, 50, 50, 60, 60, 70, 70, 80,
              80, 90, 90, 100, 100, 110, 110, 120, 0, 0, 120, 120],
             [0, 0, 53.3, 53.3, 0, 0, 53.3, 53.3, 0, 0, 53.3, 53.3, 0, 0, 53.3,
              53.3, 0, 0, 53.3, 53.3, 0, 0, 53.3, 53.3, 53.3, 0, 0, 53.3],
             color='black')

    if fifty_is_los:
        ax.plot([60, 60], [0, 53.3], color="gold")
        ax.text(62, 50, "<- Player Yardline at Snap", color="gold")
    # Endzones
    if endzones:
        ez1 = patches.Rectangle(
            (0, 0),
            10,
            53.3,
            linewidth=0.1,
            edgecolor="black",
            facecolor=ez_color,
            alpha=0.6,
            zorder=0,
        )
        ez2 = patches.Rectangle(
            (110, 0),
            120,
            53.3,
            linewidth=0.1,
            edgecolor="black",
            facecolor=ez_color,
            alpha=0.6,
            zorder=0,
        )
        ax.add_patch(ez1)
        ax.add_patch(ez2)
    ax.axis("off")
    if linenumbers:
        for x in range(20, 110, 10):
            numb = x
            if x > 50:
                numb = 120 - x
            ax.text(
                x,
                5,
                str(numb - 10),
                horizontalalignment="center",
                fontsize=20,  # fontname='Arial',
                color="black",
            )
            ax.text(
                x - 0.95,
                53.3 - 5,
                str(numb - 10),
                horizontalalignment="center",
                fontsize=20,  # fontname='Arial',
                color="black",
                rotation=180,
            )
    if endzones:
        hash_range = range(11, 110)
    else:
        hash_range = range(1, 120)

    for x in hash_range:
        ax.plot([x, x], [0.4, 0.7], color="black")
        ax.plot([x, x], [53.0, 52.5], color="black")
        ax.plot([x, x], [22.91, 23.57], color="black")
        ax.plot([x, x], [29.73, 30.39], color="black")

    if highlight_line:
        hl = highlight_line_number + 10
        ax.plot([hl, hl], [0, 53.3], color="yellow")
        ax.text(hl + 2, 50, "<- {}".format(highlighted_name), color="yellow")

    border = patches.Rectangle(
        (-5, -5),
        120 + 10,
        53.3 + 10,
        linewidth=0.1,
        edgecolor="orange",
        facecolor="white",
        alpha=0,
        zorder=0,
    )
    ax.add_patch(border)
    ax.set_xlim((0, 120))
    ax.set_ylim((0, 53.3))
    return ax


# game_play = "57783_003374"
game_play = ['_'.join(x.split('_')[:2]) for x in sample_videos][0]
example_tracks = tracking.query("game_play == @game_play and isSnap == True")
ax = create_football_field()
for team, d in example_tracks.groupby("team"):
    ax.scatter(d["x"], d["y"], label=team, s=65, lw=1, edgecolors="black", zorder=5)
ax.legend().remove()
ax.set_title(f"Tracking data for {game_play}: at snap", fontsize=15)
plt.show()

In [ ]:
# Plotly Field
import plotly.express as px
import plotly.graph_objects as go
import plotly


def add_plotly_field(fig):
    # Reference https://www.kaggle.com/ammarnassanalhajali/nfl-big-data-bowl-2021-animating-players
    fig.update_traces(marker_size=20)
    
    fig.update_layout(paper_bgcolor='#29a500', plot_bgcolor='#29a500', font_color='white',
        width = 800,
        height = 600,
        title = "",
        
        xaxis = dict(
        nticks = 10,
        title = "",
        visible=False
        ),
        
        yaxis = dict(
        scaleanchor = "x",
        title = "Temp",
        visible=False
        ),
        showlegend= True,
  
        annotations=[
       dict(
            x=-5,
            y=26.65,
            xref="x",
            yref="y",
            text="ENDZONE",
            font=dict(size=16,color="#e9ece7"),
            align='center',
            showarrow=False,
            yanchor='middle',
            textangle=-90
        ),
        dict(
            x=105,
            y=26.65,
            xref="x",
            yref="y",
            text="ENDZONE",
            font=dict(size=16,color="#e9ece7"),
            align='center',
            showarrow=False,
            yanchor='middle',
            textangle=90
        )]  
        ,
        legend=dict(
        traceorder="normal",
        font=dict(family="sans-serif",size=12),
        title = "",
        orientation="h",
        yanchor="bottom",
        y=1.00,
        xanchor="center",
        x=0.5
        ),
    )
    ####################################################
        
    fig.add_shape(type="rect", x0=-10, x1=0,  y0=0, y1=53.3,line=dict(color="#c8ddc0",width=3),fillcolor="#217b00" ,layer="below")
    fig.add_shape(type="rect", x0=100, x1=110, y0=0, y1=53.3,line=dict(color="#c8ddc0",width=3),fillcolor="#217b00" ,layer="below")
    for x in range(0, 100, 10):
        fig.add_shape(type="rect", x0=x,   x1=x+10, y0=0, y1=53.3,line=dict(color="#c8ddc0",width=3),fillcolor="#29a500" ,layer="below")
    for x in range(0, 100, 1):
        fig.add_shape(type="line",x0=x, y0=1, x1=x, y1=2,line=dict(color="#c8ddc0",width=2),layer="below")
    for x in range(0, 100, 1):
        fig.add_shape(type="line",x0=x, y0=51.3, x1=x, y1=52.3,line=dict(color="#c8ddc0",width=2),layer="below")
    
    for x in range(0, 100, 1):
        fig.add_shape(type="line",x0=x, y0=20.0, x1=x, y1=21,line=dict(color="#c8ddc0",width=2),layer="below")
    for x in range(0, 100, 1):
        fig.add_shape(type="line",x0=x, y0=32.3, x1=x, y1=33.3,line=dict(color="#c8ddc0",width=2),layer="below")
    
    
    fig.add_trace(go.Scatter(
    x=[2,10,20,30,40,50,60,70,80,90,98], y=[5,5,5,5,5,5,5,5,5,5,5],
    text=["G","1 0","2 0","3 0","4 0","5 0","4 0","3 0","2 0","1 0","G"],
    mode="text",
    textfont=dict(size=20,family="Arail"),
    showlegend=False,
    ))
    
    fig.add_trace(go.Scatter(
    x=[2,10,20,30,40,50,60,70,80,90,98], y=[48.3,48.3,48.3,48.3,48.3,48.3,48.3,48.3,48.3,48.3,48.3],
    text=["G","1 0","2 0","3 0","4 0","5 0","4 0","3 0","2 0","1 0","G"],
    mode="text",
    textfont=dict(size=20,family="Arail"),
    showlegend=False,
    ))
    
    return fig



tracking["track_time_count"] = (tracking.sort_values("time").groupby("game_play")["time"].rank(method="dense").astype("int"))

fig = px.scatter(
    tracking.query("game_play == @game_play"),
    x="x",
    y="y",
    range_x=[-10, 110],
    range_y=[-10, 53.3],
    hover_data=["player", "s", "a", "dir"],
    color="team",
    animation_frame="track_time_count",
    text="player",
    title=f"Animation of NGS data for game_play {game_play}",
)

fig.update_traces(textfont_size=10)
fig = add_plotly_field(fig)
fig.show()

In [ ]:
tracking.loc[tracking['isSnap'] == True]